# lesson 6 configure azure sql db for scale and performance

**note:backup storage redundancy** 

geo redundant backup will replicate the storage to a paired region (e.g. US East to Us West) in case of data storage damaging

**note:billing type** 
- provisional charged based on vcore
- serverless charged based on second, shut down after 1 hr inactivity, restarted takes a little time


**note:hybrid benefit** 
- it's a Microsoft licensing program that allows customers to use their existing on-premises Windows Server and SQL Server licenses with active Software Assurance or subscription to reduce costs when running those workloads in Azure
- can only work with vcore, not with dtu
**note:elastic pool** 

- let manage multiple db resource allocation possible, so they could share resource when one not using it
- not possible for hyperscale
- can select edtu (elastic dtu) in setting
- unit price for edut compared to dtu is extra 50 %, vcore have same unti price
- if a DTU db peak at different time, elastic pool could be a good idea, otherwise it's more expensive due to unit price
- unavailable at hyperscale




### storage purchasing model
- can be changed later, except changed out of hyperscale
- change should be done at a low trafic time
- after change DMV(dynamic management view) needed to be flushed for accurate estimation
exec sq_query_store_flush
##### DTU (database transaction unit)based
- DTU can be adjusted with related S tier
- CDC (changed data captured) can't be used when having less than 100 DTB (= S3 = 1vcore)
- when using more than 300 DTU, vcore might be a cheaper option
- plan selection can based on peak usage and storage need, the latter could be estimate with average data use * db number 
1. Basic: up to 2GB
Development, testing, infrequently accessed workload
2. Standard
3. Premium
suitable when more iops (inputs output per second) is needed
##### vCore (virtual core) based
- max data storgae and core could be specified
- core decide max data storgae ceiling & tempdb size
- log space is 30% of the max data stoarge
- hardware choice affect memory, core number and storage
- iops = concurrent number of request, this can't be configured, but go up with vcore

4. general purpose: 80 vcore, 4 terabyte storage
suitable for low latency input
5. Hyperscale: 80 vcore, 100 terabyte storage
6. Business critical: 80 vcore, 4 terabyte storage, fast geo-recovery and advanced data corruption protection, free second read-only replica
Business critical has high transaction rate and high resiliency, is suitable for large number/long-running transaction

### table partitioning

- a table have 3 elements storage space, computing resource limit (exceeing might lead to timeout) and network bandwidth limit
- vertical scaling by adding disc space, processing power, memory and network connection could be a short term solution
- horizontal scaling by divifing the table up according to rule e.g. by year, it add up complexity but reduce hardware demand
- back up per partition is also faster 
- the other option is to split the table columns up

### database  sharding
- when we apply horizontal scaling on db level
- there would be a table with shard key, and where they are stored, this require more setting up 
- range strategy is diving by range e.g.years, sequential shard, data might be unblanced when there is seasonality
- hashing strategy introduce random element for distribution, it reduce hotspot bcs no cluster would be more used, but rebalancing and manging mmight be difficult, when looking up a value the results are all over places
- functional partitioning, not putting all the tables in the same db, e.g. seperate tables that needs extra security
- keeping data geographically close can reduce latency when using

**note: file group**
- can't be used in Azure SQL db (only primary file group there)
- file can be putted in file group to define partition range rules and where to store

### compression for table
- pro reduced space
- con takes extra power and time to compress and retrieve 
- can't be used on table with sparse columns
- changing compression require droping clustered index
- complete index viewed can be compressed
- system table can't be compressed
- table with different partition can be compressed differently
- *exec sp_estimate_data_compression_savings* help estimate compression effect

method:
1. no compression
2. row compression: char and nchar columns would be greatly reduced, varchar and int not so much, date types depends, tiny int not at all
3. page compression which includes (below element can't be activated seperately):
    1. row compression
    2. prefix compression: store long duplicated value in a specific column as prefix key, and generate an extra table for prefix key to long name to prevent the duplication of long char storage
    3. dictionary compression: similar to prefix compression, but instead of on a specific column, the duplication would be searched across columns and keys would be used for whole table

    - page is a set of row up to 8192 character
    - uncompressed at first, when additional rows can't be fit in, compression takes place
    - some older version SQL server on VM is this unavailable

**note heap**
haep is table without clustered index

**columnstore table**
above are row stored table. in columnstore table, columns are always compress, which could be further compressed through columnstore archival compression (but this has super high compute cost for uncompress)

### SQL data sync
- works for Azure SQL, on-prem or VM, but not with management instance
- allows sync across db
- primary key is required
- must define a hub db and multiple member dbs
- rules can be defined which direction to sync, e.g. change would be replicated from hub to member or member to hub, but not directly between members
- rule when hub and member simultaneously changed should be defined
- sync metadata db is an azure sql db whihc loacte in the same region as hub and store, should be empty at start
- for on perm or vm member db, a sync agent programm is needed 
- on azure db page syncing db could be created


# lesson 7 strategy of migration 

### migration tools

| purpose    | tool | extra function |
| -------- | ------- | ------- |
| lift & shift sql server to vm   | azure migrate    |   assess sql data estate at scale(across data center)/getting azure sql deployment recommendation, target sizing & monthly estimate  |
| non sql objects (DB2, MYSQL, Oracle, SAP, ASE) to sql/azure sql, each are independent download program | sql server migration system     ||
| sql server object to sql db/managed instance/vm/on prem     | data migration system    |upgrade SQL/assess sql data estate at scale/ performance & reliability improvement recommendation/ detect incompatibility of target/ migrate on-prime/ discover new feature on target|
| compare workloads between source and target  | database experiementation assistant    | |
| open source db (mysql, postgresql, mariadb) to azure offline/online (premium price tier)  | azure db migration service    ||

1. azure migrate  
- consists of 2 tools: discovery & assessment, server migration
**azure migrant hub** includes azure migration assistant & azure db migration service

2. sql server migration system 

3. data migration system
- for large size data azure db migration service  would be better option

4. database experiementation assistant 

5. azure db migration service 
- princing tier standard provides up to 4 vcore for free, but only support offline migration
- princing tier premium provides up to 4 vcore with 6 month free trial, support offline & online

| from    | to | online (continuous sync)    | offline (one-time) |
| -------- | ------- | -------- | ------- 
| SQL server  | azure sql db    | x  | v    |
| SQL server    | azure sql db mi    | v | v    |
| SQL server    | azure sql vm  | x  | v    |
| mongodb    | azure cosmos db  | v | v    |
| mySQL    | azure db for mySQL  | x  | v   |
| postgreSQL    | azure db for postgresql  | v  | x  |




**reminder** if not latest SQL is needed, then VM must be used and a window version needed to be decide

**note** 
- without downtime allowance, the only way is to do an online migration
- if cross db dependecy exist, azure db is not a good option, as it doesn't allow cross db queries
- for azure db migration service, allow outbond port 443 = port for https or 1434 used for UDP, then enable TCP IP protocal, next create azure SQL db instance which needs a server level firewall rule on depature db/server & contorl server permission on depature server to allow access to DMS and on target db control db permission is needed. it use exsiting full log backup instead of creating new one
- there are other service e.g. Bulk copie programm, command bulk insert, log data from Azure blob storage, SQL server integration service package, spark/azure factory
- migrant stuff directly between Azure SQL db can use export or export data tier application (which allow export of schema), or export form azure portal, download and using SQLpackage
- export data tier application (DAC) will make the export as .bacpac (backup package) file

### post migration
1. have tests to make sure data are there
2. checking Query returns same result in sourece and target db
3. checking the performance
4. changing the app connecting to db from departure db to target db
5. checking missin features

# lesson 8 Microsoft Entra ID


### authn = authentication = log in
1. window authentication
    - on a window machine/domain 
    - in azure only on VM is window available
2. SQL sever authentication
    - using username and password
3. azure active directory
    - setting could be access through azure portal, e.g. adding user
    - more secure than SQL sever authentication and can select way to authenticate in portal 
    - once the AAD admin is setted through azure portal (only 1 admin allow), the admin can log in SSMS through (other log in method won't allow role creation) and create more roles through query 
    - comparison between AAD admin and sql server admin:
        - both could create users based on SQL server authentication log in
        - both can create contained DB users based on SQL server authentication wo log in
        - AAD admin can create contained DB users based on aad users and group
    - authentication methods includes FIDO secutity keys which is a hardware on computer, microsoft authenticator, text message and temporary access pass
    - can sync with on-prem windows server active directory
    - when we are logging in azure portal is a form of azure active directory
    - azure active directory - integrated/universal with MFA(multi factor authentication) can be used when using admin tool like SSMS on a computer that is not domain joined 
    - 3 different way to authenticate in:
        1. cloud only identity
            - handle sign in completely in cloud by azure
        2. federated authentication
            - integrate with existing federation provider
            - sign in requirement which is not native supported by Azure active directory
        3. pass-through authentication
            - when don't have a sign-in requirement not natively supported by Azure active directory
            - and want to enforce extra feature like user level active directory sign in policy
    - identity: 
        1. cloud only identity
        2. hybrid identity support both cloud authentication & pass-through authentication
        3. hybrid identity support federated authentication
    - to create a user based on AAD user use: create user [user_email] from external provider
4. window server active directory

**note:**
- authz = autherization = what one could access

### role
-  when running on an non-azure machine that is domain joined by creating a certificate and connect the app to azure data
- to see all role in azure SQL use sp_helprole
- after creating user, adding roles for the user is needed in SQL server
    - alter role [role name] add/drop member [member email]
    1. db_owner : have access to all.
    2. db_securityadmin : modify role membership for custom roles only. manage permission, can elevate own permission
    3. db_accessadmin: add & remove access to db for login & groups
    4. db_backup_operator: not applicable in azure. can backup db, mi & vm
    5. db_ddladmin: can run ddl comment (creat, alter, drop)
    6. db_data_reader: can read all data
    7. db_deny_data_reader: can't read old data
    8. db_data_writer: ability to add, delete or change data
    9. db_deny_data_writer: can't add, delete or change data
    - on-prem only, roles that exist in master db
    10. db_manager: create/delete db
    11. login_manage: create/delete login in master db = security_admin for master db
- role based access control (RBAC) in azure:
    - Access control section is called IAM, when role could be added
    - there is no access to server wide logging permission bc access to underlying server doesn't exist
    1. SQL DB contributer: allow SQL DB, but don't have access to content
    2. SQL server contributer: allow SQL server/security policy manage, but don't have access to content
    3. SQL security manager: allow SQL security policy manage, but don't have access to content

### granting permission
- granting access to particular schema/table can't be achieved through these set roles in stead grant permission command should be used
- command: authorization permission on securable::name to pricipal with grant option
- select * from sys.fn_my_permissions(null, 'DATABASE') shows all permission
1. autherization (grant, revoke, deny)
    - grant [permission_type] on object::[schema].[table] to [user_email]
    - to revoke the permission used revoke [permission_type] on object::[schema].[table] to [user_email]
    - to ban someone from table access that cannot be overwrite by grant use deny [permission_type] on object::[schema].[table] to [user_email]
2. permission 
    - table: select, insert, update, delete, control(all right to table), reference(view foreignkey), take ownership(change ownership), view change tracking, view definition (rightclick and go to script table path)
    - schema: alter(=alter/create/drop any securable)
    - function/stored procedure: alter, execute(even lack of select permission on table is not a problem, this is called ownership changing), view change tracking, view definition
3. securable (object, schema, db, server)
4. principal (login, user, role: create role [rolename], add member [useremail])
    - public is a special role that every single user of the db have this role
5. with grant option: allow the principal to share the permission later

**note:apply principal of least privilige for all securable**
a user should have least privilige user account (LUA)


# lesson 9 implement security

### transparent data encryption (tde)
- this is the security for data in transit
- it encrypts then deencrypts data at page level at rest
- encrypts at write then decrypts at read
- for azure SQL db this is set at server level, new db inheritate the rule unless recover from a copy, encryption can be turn off in azure portal / Tsql / cloudshell / restapi
- for azure sql mi this is set at instance level,  new db inheritate the rule but not system db, encryption can be turn off  in Tsql/ cloudshell/ restapi
- it use database encryption key (dek), which is a symetric key = one single key is needed for en- & decrypts
    - this could be a service managed key, protected by a TDE protector using service managed certificate 
    - this could be a bring your own key, in this case it could also be asymmetric

**note** transparent layer security (TLS), encrypts when in transit

### object level encryption (e.g. table with sensitive data)
1. always encrypted
    - always encrypted is available in azure sql db, mi, vm
    - this could be done in ssms through right click encrypt columns
    - encryption type has deterministic (encryption is the same everytime, allows for join, group by, index, distinct without decrypt) & random 
    - when the column is used as index, it can't be randomized encrypt
    - to create encryption, key vault is needed
    - after creation, next time when connecting to db in SSMS select enable alway encrypt 
    - a pricipal (e.g. user) need to be in key vault to decrypt encrypted colunm

    - to query an encrypted column in where clause, set up a parameter & enable parameterlization for always encrypted is needed, but this will still block funtion like
    - to enable partial string comparison, always encrypted need to be used with secure enclaves
        1. enable confidential computing through configured as DC series
            - strongest security isolation
            - type name is intel software guard intellegence(sgx)
            - require ms azure attestation
        2. enable secure enclaves in a standard configure db
            - type name is virtualization based security (vbs)
            - available for all ay sql, and sql 2019+
            - not available in ger india central
            - can't defend against e.g. replacing enclaves program with malware
            - creating a protected part of memory within which data is display as plain text
            - protection against operative system threat
            - standard azure protection (e.g. multifactor authentication, just in time access)
            - can't be deactivated after activation

    - permission wise there are
        1. alter any column master key
        2. alter any column encryption key
        3. view any column master/encryption key (for access, query and read encrypted data)
    - role wise there are
        1. security admin who generate column encryption key and master key, access to key and key store are required, access to db is not required
        2. db admin who manage metadata about key, access to key and key store isn't required, access to db is required
        - if these 2 role should belongs to 2 ppl, then powershell should be used
        - if these 2 role should belongs to same ppl, then powershell or ssms could be used

2. dynamic data masking
    - this could be done through azure portal 
    - in tsql: alter table [schema].[table] alter column [column] add masked with (fuction = 'default()')
    - it can be apply on specific column with desired partial masking 2025-0x-1x
    - specific users could be excluded in azure portal/tsql: grant(/revoke) unmask to [useremail] 
    - this could also be done through powershell
    - mask type
        - default value 0
        - number: choose random number between 2 different bundary
        - email: expose first value of email
        - credit card: last 4 digits of card number
### key vault 
- could be created in azure sql portal
- in key vault store secrets
- key vault name need to be unique
- better store it in region near db
- pricing tier premium allow use of hardware security model (HSM). it's the same as standard  until hsm is used
- in access policy selecting unwrap key + wrap key + verify + sign (and also the default selections) is necessary to column create primary key 


| | always encrpted    | transparent data encrpytion (tde) |
| -------- | ------- |------- |
| sql server version  |  2016+   | 2008+ |
| sql server entriprise edition | x     | v|
| free in azure sql db    | v    | v|
| protect data at rest   | v   | v |
| protect data in use    | v  | x (when in transport, using transport layer security) |
| protect data from sql admin  | v  | x |
| data is de/encrypted on the client side  | v | x|
| data is de/encrypted on the server side   | x | v |
| encrypt at    | column level  | entire db|
| transparent to application   | partially | v |
| encryption option   | v | x |
| key management   | customer managed keys | server/customer managed keys |
| protects key in use  | v | x |
| driver required | v |  x|


### configure server/db level firewall
- by default all connection are rejected, unless firewall is set up
- db rule would be checked before server rule when implement
- server rule need to be set before db rule
- doesn't apply to azure sql mi





|| server   | db |
| ------- |------- |------- |
|| for user and app to have access to all dbs in the server| for an individual/app |
|can be set in azure portal|v | x|
|role|sql server contributor/sql security manager/owner of the resource | control db permission is required at db level|
|tsql to check rules|select * from sys.firewall_rules  | select * from sys.database_firewall_rules |
|tsql to set rules|execute sp_set_firewall_rule @name=N'rule', @start_ip_address='ip', @end_ip_address='ip' (need to be in masterdb)| execute sp_set_database_firewall_rule @name=N'rule', @start_ip_address='ip', @end_ip_address='ip' |
|tsql to delete rules|execute sp_delete_firewall_rule @name=N'rule' (need to be in masterdb)| execute sp_delete_database_firewall_rule @name=N'rule'|

**note** n before quote change varchar to nchar/nvarchar

### transport layer security (tls)
- encrypt data on db/client side and decrypt it from other side
- most widely used is TLS 1,2, available in 2008 and is used in azure sql db
- if the tls version from cleint older than min version allowed in target db , connection would fail
- min tls version can be set in powershell/azureportal






# lesson 10 & 11 implement compliance control for sensitive data

### data classification
- sensitive data include data privacy, regulatory and national requirement
- classification can be set in azure portal, it will also show recommanded classification 
- information type and security label can't both be n/a
- security label:
    - n/a : available to all 
    - public: public released data
    - general: data not mean for public e.g. emails, docs
    - confidential
    - confidential - GDPR
    - highly confidential
    - highly confidential - GDPR
- role to change classification: db_owner, db_contributor, sql_security_manager
- role to read classification: reader, user access admin
- tsql to check: select * from sys.sensitivity_classifications
- tsql: select * from sys.columns where object_id = [majorid from above query]
- tsql to add: add sensitivity classification to [schema].[table].[column name] (with label='label name', information_type='info name', rank=low/medium/high/critical/none)
- tsql to drop: drop sensitivity classification to [schema].[table].[column name] 

### configure server and db audits
- audit retain trails of selected db actions, report db activity using preconfigured report on dashboard and analyse report for suspicious event
- not supported for premium storage/hierarchical namespace
- having server and db audit at the same time is possible, but only using server level is recommand, unless specific requirement for specific db is needed
- default policy includes batch completed group, all the queries & storage procedure, successful db & failed db authentication group (e.g. login)
- it store around 4000 character in an audit 
- server policy audit always apply to db disregarding db level audit policy
- audit log storage:
    1. existing/ new storage account
    2. existing monitor log analytics workspace
    3. existing event hub
- audit log can be viewed in azure portal or ssms merge audit file and add from azure blob storage

### data change tracking
- both can be used in azure sql db, but only change data capture can be usedd in aure sql mi
1. change tracking (ct)
    - when a row/column get changed it will be tracked
    - doesn't track how many time it has changed/history data
    - it store less data than adc
    - it store data in an in-memory roaster and flushed on every checkpoint to the internal data (it kept in memory and then saved often)
    - snapshot isolation
        - SQL Server recommends using snapshot isolation with change tracking
        - Snapshot isolation ensures that the change tracking process itself operates on a consistent snapshot of the data, preventing issues where changes might be missed or incorrectly reported due to concurrent modifications 
        - using snapshot isolation will makes any changes while geting data invisible (it has a set of data instead of a changing set)
        - tsql for snapshot isolation:ALTER DATABASE [dbname] SET ALLOW_SNAPSHOT_ISOLATION ON then SET TRANSACTION ISOLATION LEVEL SNAPSHOT
    - enable ct can be done in ssms /tsql, it has to be done first on db level then for specific tables
    - for tracking updates command, extra parameter need to be set
    - tsql for enabling ct: alter database mydb set change_tracking = on (change_retention = 2 days, auto_cleanup = on)
    - to disable, all tables need to be disabled then the db
    - tsql for Check which tables/databases have CT enabled: 

        SELECT * from sys.change_tracking_databases 
        select * from sys.databases 
        SELECT * from sys.change_tracking_tables  
        select * from sys.objects 
    - tsql for getting the initial sync version:

        DECLARE @last_sync bigint; 
        SET @last_sync = CHANGE_TRACKING_CURRENT_VERSION();  
        select @last_sync 
        select CHANGE_TRACKING_CURRENT_VERSION() 

    - tsql for finding out what after changes have happened: 

        SELECT  CT.AddressID, CT.SYS_CHANGE_OPERATION,   
        CT.SYS_CHANGE_COLUMNS, CT.SYS_CHANGE_CONTEXT 
        FROM  CHANGETABLE(CHANGES SalesLT.Address, 0) AS CT





2. change data capture(cdc)
- not available in basic price tier, available in standard price tier from s3
- need to first enable for db then for table
- tsql for Enable for database (can't be used in system db) :
    EXEC sys.sp_cdc_enable_db  
- tsql for Enable for table :

    EXEC sys.sp_cdc_enable_table @source_schema = 'SalesLT', @source_name = 'Address', 
    @role_name = 'NewRole', 
    @captured_column_list = 'AddressID, City'

- tsql for checking configuration:

    EXECUTE sys.sp_cdc_help_change_data_capture 

- tsql for Disable from table :

    EXEC sys.sp_cdc_disable_table @source_schema = 'SalesLT', @source_name = 'Address', 
    @capture_instance = 'all' 

- tsql for Disable for database :

    EXEC sys.sp_cdc_disable_db 

- tsql for checking changes:

    DECLARE @from_lsn binary(10), @to_lsn binary(10);   
    SET @from_lsn = sys.fn_cdc_get_min_lsn('SalesLT_Address');   
    SET @to_lsn = sys.fn_cdc_get_max_lsn();   
    SELECT * FROM cdc.fn_cdc_get_all_changes_SalesLT_Address  (@from_lsn, @to_lsn, N'all')

    **note** lsn = log sql number

    **note** cdc.fn_cdc_get_all_changes_SalesLT_Address is a custome sql function made by cdc, it has name of the specific table, argument are from, to , and display type

### vulnerability assessment
- a defender for Servers plan is required to do assessment, azure defender cost 15 dollar per month per server
- adding a failed test as baseline will change the failing status

### azure purview
- when data is stored in different places it's hard to check for compliance
- purview catalogues the data regardless of on premises, in a machine, online, on cloud using ssas
- accessing new data source from purview
    1. azure service need to allow connection from azure service in server setting for purview to work
    2. a secret (e.g. sql password) in keyvault (using purview resource group) need to be added
    3. in access policy from key vault give permission to purview, with key permission get & list
    4. in purview credential, add the secret from step 2
- using purview
    1. create ms purview account (need subscription & resource group )
    2. open ms purview portal which shows 3 main element:
        1. data map: matadata created through classifying and scanning varios sources
        2. data catalogue: help with finding data with classification/metadata filter
        3. data insight: locate sensitive data
    3. using scan rule set which help group together classification & file type
- collection & scan 
    - collection within collection is allowed
    1. add a source & selecting which source it should be linked to
    2. click scan on the added source, define scan table, targeted data type & time schedule
    - lineage extraction is limited to store procedure run, otherwise should be turned off


### azure db ledger

- ledger provides a history of changes made to a database's data, ensuring data integrity and enabling auditing

**what is updatable ledger table**
- updatable ledger table provide cryptogrphic (a security communication technique) proof of date to the auditors. it reduce time to audit data. 
- all the modification that is done to updatable ledger table is hashed cryptographically using SHA256 (= a root hash). 
- a hash is similar to a footprint that is left behind by the footstep. root hash are stored in blocks. blocks are closed after 30 sec/ 100,000 transactions, then hashed along with a root hash pf ürevious block, forming a blockchain0.
- the latest hash block is called database digest
**how updatable ledger table is stored**
- db ledgers are stored in trusted storage e.g. immutable azure blob storage(which follows write one read many times = can't be altered), azure credential ledger
- data integrity can be verified by comparing db digest hash against calculation of current table
- sql db manage db ledger transparently
**how updatable ledger table created**
- non ledger table can't be converted into ledger table
- data need too be copied into new ledger table(select into/bulk insert/sp_copy_data_in_batch, sp=stored procedure), then rename ledger table as original table name
    - benefit of sp_copy_data_in_batch is it split copy operation into batches, 10 - 100 k per transaction done in parallel, therefore speed up the process
1. in ssms go to table, right click, script table as
2. copy the script to create a new table, with new name and adding tsql
    for updatable: with (system_versioning = on, ledger = on)
    for append only: with (ledger = on,append only = on)
3. insert everything from original table into this ledger table
4. now the history table can be seen in sys.table if updatable is used
5. could query from the ledger view

**what's in updatable ledger table**
- ledger table have 4 new cols, named as always generated columns, automaticallly generated 
    1. ledger_start_transaction_id: unique transaction id of each inserted action, e.g. if 10 rows are inserted at the same time, they'll all have the same id
    2. ledger_end_transaction_id
    3. ledger_start_sequence_id: e.g. if 10 rows are inserted at the same time, they'll have sequence from 0 to 9. for next transaction sequence starts from 0 again
    4. ledger_end_sequence_id

**what is history table**
- it's also built on db ledgers
- it shows previous version of a row, when it's updated/deleted in the updatable ledger table
- it has also 4 always generated columns
- data can't be deleted from here
- automatically named as original table name.mssql_ledgerhistoryfor_guid

**what is ledger  view**
- it join updatable ledger table and history table
- it shows transaction id, delete/insert/update(update is both delete & insert)
- is recommanded to used instead of the other 2 tables

**append-only ledger table**
- another version of ledger table, - it's also built on db ledgers
, alternative to the updatable ledger table
- data can be insert but not update/delete
- in this case history table is not needed
- there is still a ledger view, provide info on transaction & user who inserted data
- 2 always generated columns
    1. ledger_start_transaction_id: 
    2. ledger_start_sequence_id: 

**verifying**
1. in azure portal, select digest storage, adding a storage container
2. go to verify db option, which provides code to look at data_ledger_digest_location 
    - storage container could also be activated in security tab when creating azure sql db
    - when creating azure sql db if select ledger db, then all the table in this db will by default all and only be ledger tables
3. copy this and run in ssms, it'll show:
    - digest location (path of the digest)
    - last verified blog id 
    - whether the path is the latest location, if unsuccessful then db has been tempered with
4. if it's been tampered with restored to a verifiable version is needed, then manually create any future transaction while using backups after the point of time


### row level security
- available for sql 2016 +
- actions like select, update and delete could be restricted to specific rows
- block write operations on new/existing data: after insert/after update
- block update/delete on existing data: before delete/before update
1.  it's recommanded to have a seperate schema for row level security object, it helps reducing effort in maintaining various permission 
2. create inline table valued function (tvf), it returns when user should see the result: 

    create function rls.rls_security(@user an nvarchar(10)) returns table with schema binding 

    as return select 1 as result 

    where @user = username()
3. add select permission to the tvf: 

    grant select on rls.rls_security to [user1] 

    grant select on sceham.table to [user1]
4. create security policy (role needed): 

    create security policy policyname 
    
    add filter predicate rls.rls_security(usercolname )
    on sceham.table

    **note** this filter out what shouldn't be seen

    add block predicate rls.rls_security(usercolname )
    on sceham.table after insert

    **note**  block insert outside of what allowed to be seen

    with (state = on)

    go
- without turining the security policy on, everyone will be able to view all rows, after turning on, only the user being granted access can

### configure advanced threat protection
- available for azure sql db, mi, vm
- it allow admin to be alerted when potential threat happened e.g. suspicious pattern on querying/access, possible vunerability, sql injection attack
1. enable ms defender for cloud
2. click on the configure and set up email notification for advanced threat protection alert
3. the email will also show the suggested action for fixing

# lesson 12 monitoring activity and performance

### creating performance base line
- performance are affected by hardware (pricing tier)
- on vm performance is also affected by operation system, db applications and client application
- with a high dtu percentage(= processor percentage, Database Transaction Unit, it represents a blended measure of CPU, memory, reads, and writes) or i/o percentage :
    1. more cpu/input output resource is needed 
    2. query needs to be optimized
1. metrics explorer

    - metrics are collected at regular interval and stored in a time series db
    - they have a timestamp for when, name, value and other label
    - it's light weighted and allows for near real time allerting
    - in monitoring section, metric tab allows one to see metric for a specific sql db
    - max 30 days at once 
    - up to 93 days in the past is viewable

2. log
    - operation performance metrics can also be created by using log
    - logs can be structured or free form, may have a timestamp
    - in monitoring> logs> diagnostic> loading data, last hour logs can be imported using language kusto

3. tsql
    - SELECT * from sys.dm_db_resource_stats  
        - current db or elastic pool CPU, IO and memory 
        - a row every 15 s, for last 1 hr
    - SELECT * from sys.server_resource_stats  
        - equivalents for reports on azure sql mi, CPU, IO and memory 
        - a row every 15 s, for last 1 hr

    - SELECT * from sys.resource_stats 
        - reports on server( multiple db), CPU, IO and memory 
        - a row every 15 s, for last 1 hr

    - SELECT * from sys.dm_user_db_resource_governance  
        - Returns one row actual configuration and capacity settings used by resource governance mechanisms in the current database or elastic pool
        
    - SELECT * from sys.resource_usage  
        - provides hourly summary of result usage data for user db in current server
        - even if db is idle, as long as it's accessible, it'll have 1 row/hour 
        - the view doesn't contain usage anymore, only storage in megabyte
    - exec sp_who 
        - show current user , process, whether it's blocked
    - exec sp_lock 
        - show lock, object id, index id
        - Locks are held on SQL Server resources, such as rows read or modified during a transaction, to prevent concurrent use of resources by different transactions   
    - exec sp_spaceuser
        - display estimation of current disc space use     by a table or db 
    - exec sp_monitor
        - work for on prem db but not for azure sql db
        - show cpu usage, io usage, idle time etc

**note** An on-premises database, refers to a database system that is hosted within a company's own facilities, typically in their own data center or server room

### create event notifcation
- alert could be created from alert, metric or log tab in azure portal
- how frequent the measure being group together and how frequent the alert rule being checked need to be selected when creating the rule, if grouped every 30 min, checking frequency can't be lower than every 30 min
- static threshold would be based off of a fixed figure
- dynamic threshold would be automatically decided by machine, taking into account factors like seasonality
    - in dynamic mode how sensitive to the standard and how many number of violation within a arbitrary time period to trigger the alert, notice this should be a reachable goal with the checking frequency, default value is medium sensitive
- then action and notification type could be selected for the alert
- automatically resolve alert mean when value gets back to normal, resolve the alert automatically


### source for performing metrics
1. Azure tenant
    - tenant wide service: azure active directory, 
2. azure activity log: include self service health record, configuration changes
3. azure service health
4. azure resources
    - submit platform metrics to metrics db
    - reource log then create internally regarding internal operation of resources
    - for guest operating system, azure dignositc extension for vm if enabled or log analytic agent that can install on machine or vm insight


### interpreting performace metrics
- system dmdb resource stats: if any of that is close to 100 %, then upgrade the service plan might be needed, if too low downgrade can save money
- xtp storage percent: percentage of storage used by In-Memory OLTP objects in Azure SQL Databas
    - 0 if in memeory optimized table not used
    - 100 update and insert will fail, select and delete are fine
- max session percent: the max concurrent session allowed/service tier limit
- max worker percent: max concurrent request/service tier limit
- in intelleigence performance section in azure portal, in query performance insight and see the most heavy query by cpu/data io/ log io, some of them come with performance recommendation

### intelligent insight

- it compares current db workload to last 7 days performance
- looks for things that could affect db performance e.g.resourcing limit, worklaod increase, memory pressure (workers waiting for memory allocation), data locking or need to increase maximum degree of parallelism = dop, missing index, new query affected performance, multiple thread using same temp db resource
- automatically enable for azure sql db, need to manual enable for azure mi
- ai detect high wait time/query prioritization/
- not available for vm and some europe/us region
- include 
    - query performance insight 
    - diagonostic setting
        - log type includes
            - SQLinsights
            - automatic tuning
            - timeouts
        - destination, where the log can be sent to includes
            - log analytics workspace
            - storage account
            - stream to event hub
            - partner solution


| event/ activity  | extended events | sql server profiler |distributed replay |system monitor = performance monitor | activity monitor in ssms| transact sql| error log|performance dashboard in ssms|
| :----- | :------ | :------: |:-------: |:------: |:------: |:------: |:------: |-----: |
| what it is  |   light weight monitoring system, using littele resource, allow create, modify, display, analyse session data   | track process event, monitor service and db activity |sql server profiler to replay on multiple machine, simulate a mission critical workload | track resource usage, monitor server performance using counters and rates| display info of process running on sql instance|   | window app event log (for vm) | identifying performance buttleneck in sql server|
| trend analysis           | v | v |   | v |   |   |   |   |
| replaying captured event |   | v(data can be save as sql table, replay step by step from a single computer) | v(data can be save as sql table, replay step by step from a multiple computer)a| | | | | |
| adhoc monitoring         | v (xevent profiler)| v |   |   | v | v | v | v |
| generating alarm         |   |   |   | v |   |   |   |   |
| graphical interface      | v | v |   | v | v |   | v | v |
| using within custom app  | v | v |   |   |   | v |   |   |

### sql insight 
- retired at 2024 but still part of the exam
- ms allows 1/more dedicated azure virtual machine (ubuntu 18.04, size b2s) to collect info from azure sql db, mi, vm then store it in log analytics space
- sql insight/workbook template/log queries connects to this space, it's part of azure monitor
- below s1, elastic pool, some price tier plans are not supported, bcs it constantly querying the db, its not suitable for serverless
- require sql server authentication

### db watcher
- a centralized store for performance, configuration, health data for azure sql db & mi & elastic pool (* are also available for elastic pool)
    - active history
    - backup history
    - change processing and error
    - connectivity
    - (mi: db)geo replica
    - index metadata
    - memory utilization *  
    - missing index
    - out of memory event *
    - performance counter *
    - db and instance property *
    - query runtime and wait statistic
    - db replica *
    - resource utilization
    - session statistic
    - sos schedueler *
    - sql agent job can only be stored on mi
    - storage io *
    - storage utilization *
    - table metadata
- it gets data from db and store it in :
    - azure data explorer cluster: highly scalable data service for fast input and analytic 
    - microsoft fabric: realtime analytic 
- visualization is free, storage is not
- once store can be query using kusto kql or tsql in azure data explorer dashboards, powerbi, gafana, excel


# lesson 13: manage performance related maintainence task
### index maintainence task

- select * from sys.dm_db_missing_index_details 

- select * from sys.dm_db_physical_stats 
    -access fragmentation of db index
- select * from sys.dm_db_column_store_row_group_physical_stats 
    -access columnstore index
    - deleted row / total row = how fragmented index is
    - when a row in column stored get updated, it is marked as delete

-   ALTER INDEX ALL 
    ON [SalesLT].[Address] 
    REORGANIZE 
    - Reorganize indexes for fragmentation between 10-30 %, columstore for above 20&

-   ALTER INDEX [PK_Customer_CustomerID] 
    ON [SalesLT].[Customer] 
    REBUILD WITH (ONLINE = ON,  
    FILLFACTOR = 60, 
    MAX_DURATION = 30, 
    RESUMABLE = ON) 

    - Rebuild indexes  for fragmentation above 30 %
    - fullfactor allow create 135 on the page (60% filled), and leave 2 and 4 for later, the index will be bigger due to blank page
    - max_duration = time limit 

 -  ALTER INDEX [PK_Customer_CustomerID] 
    ON [SalesLT].[Customer] 
    PAUSE/ABORT /RESUME 
    - pause/abort/resume an alter

### statistic maintainence task
- use to create query plan to improve the speed, estimate cardinality(# of rows), allow seek instead of scan for query plan
- statistic includes dist of the value in table/indexed view col
- query optimizer determine update automatically, but it can also be done manually, especially if:
    - query execution time is slow
    - inserting on ascending/descending key col (e.g.identity/timestamp)
    - after maintainence operation e.g. bulk insert
- not needed to update after rebuild/reorg as rows are not changed
- EXEC sp_updatestats  
    - Update a particular table or indexed view 
-   UPDATE STATISTICS [SalesLT].[Address] [AK_Address_rowguid] 
    WITH FULLSCAN/ SAMPLE 10 PERCENT(, PERSIST_SAMPLE_PERCENT = ON) / RESAMPLE (, PERSIST_SAMPLE_PERCENT = ON) 
    - with full scan scan all rows
    - SAMPLE 10 PERCENT scan a sample
    - RESAMPLE most recent sample rate
    - PERSIST_SAMPLE_PERCENT set the value as default

### db auto tuning
- it's a process of learning about workload and identidy issue and improvement using: learn/adapt/verify/repeat
- can be configure in intellegince insight in azure portal
- for db the below 3 options are available, for mi only force plan, these setting could be inherited from server, multiple on possible
    1. force plan=force_last_good_plan, enabled by default, if estimated gain above 10s or error of new plan is above tolerence then last good plan will be used
    2. create index: auto create when cpu data io & log io are below 80 %, if performance are'nt improved, it'll be auto drop
    3. drop index: not compatiable with partition switch & index hint
-   SELECT * FROM sys.indexes  
    WHERE auto_created = 1 
    - show all index in db that's auto created
 
 -  ALTER DATABASE [DP300] SET AUTOMATIC_TUNING = AUTO --| INHERIT | CUSTOM 
    ALTER DATABASE [DP300] SET AUTOMATIC_TUNING (FORCE_LAST_GOOD_PLAN = ON, 
                                                CREATE_INDEX = ON, DROP_INDEX = OFF) 
    - Change database auto-tuning using tsql
- SELECT * FROM sys.dm_db_tuning_recommendations 


### manage storage capcity for azure sql db (may or may not applied to mi)
- data space used increase with insert, decrease with delete
- data space allocated is space made available for data, increase with data space used , but not decrease with delete
- SELECT database_name, allocated_storage_in_megabytes FROM sys.resource_stats 
    - to check allocated space for a saingle db in masterdb
- SELECT elastic_pool_name, elastic_pool_storage_limit_mb, avg_allocated_storage_percent 
FROM sys.elastic_pool_resource_stats 
    - to check allocated space for an elastic pool in masterdb
- SELECT DATABASEPROPERTYEX('DP300', 'MaxSizeInBytes') 
    - Display maximum size 


-   SELECT file_id, type_desc, size, max_size, growth 
    FROM sys.database_files 
    WHERE type = 1 
    - View current log size 
    - if size = 1000, 1000 refers to page that are still free, for each page there are 8 kilobytes, so the total available space is 8000 kilobytes
-   DBCC SHRINKFILE(file_id, megabyte_to_shrink_to) 
    DBCC SHRINKDATABASE(DP300) 
    - To shrink a transaction log file 
    - file_id megabyte_to_shrink_to are int
    - dbcc = database console command
- shrinking shouldn't be done too often as file needs to grow(?)

### assess growth/fragmentation of db/log
-   SELECT database_name, start_time, storage_in_megabytes 

    FROM sys.resource_stats  

    ORDER BY database_name, start_time
    -  Assess growth in a database for 1 database (need to be in the "Master" database)

-   SELECT start_time, elastic_pool_name, elastic_pool_storage_limit_mb, 
    
    avg_allocated_storage_percent  
    
    FROM sys.elastic_pool_resource_stats 
    
    ORDER BY start_time  

    - For an elastic pool (need to be in the "Master" database) 

- EXEC sp_spaceused 
    - Report on database free space 

-   SELECT allocated_extent_page_count, unallocated_extent_page_count  

    FROM sys.dm_db_file_space_usage 

    - View number of pages 
- DBCC SQLPERF (LOGSPACE) 
    - transaction log space statistic

- for vm in ssms disc usage can be directly viewed
- SELECT * FROM sys.dm_db_session_space_usage 
    - show # of pages for each session, internal means tempdb

- SELECT * FROM sys.dm_db_task_space_usage 
    - show # of pages for each task, internal means tempdb


# lesson 14: identify performance related issue

### query store
- query that are regressed due to change in execution plan could be fixed
- it's disabled by default on new sql db, on-perm , vm, enabled by defualt on azure sql db
- statistic collection interval can be changed, first time activation might takes day until insight coulld be produced, too often collection might affect performance
- default allocation for query store is 100 mb - 1 gb depend on sql server version
- if limit being hit, query store will not collect new data and turn into read only mode
- clean up can be automatically start when storage reach a certain percentage, it'll remove oldest/least expensive query data util back to another certain percentage
- query store capture mode, default is auto (ignore infrequent/small/quick executed query), but can be changed to all or none
- ALTER DATABASE [DP300] SET QUERY_STORE CLEAR
    - remove query store, can also be done in ui in ssms
- SELECT * FROM sys.query_store_plan  
    - extract query plan

-    SELECT Txt.query_text_id, Txt.query_sql_text, Qry.* 

    FROM sys.query_store_query AS Qry 

    INNER JOIN sys.query_store_query_text AS Txt 

    ON Qry.query_text_id = Txt.query_text_id 

    -  See the queries


1. plan store
    - for executing plan data

2. runtime store
    - for execution statistic data
3. waits stats store

- in ssms you could view regress query and seeduration, cpu time, logical read, physical read etc, in this view you could also set a particular plan for a specific query in future
- in ssms you could view query with top overall resource consumption, the view here could be customized whihc stats to see and how far back it should be
- in ssms track query you could give the query id and see and see the stats, in the view circle means completed, square means cancelled by client, triangle means failed by exception

### identify session causing blocking
- if an update is running and the the power is down halfway through, after power back on the computer will rollback the change, there is no half-complete for transaction, only success/fail, this is a lock
- explicit transaction: when tsql state begin transaction, comit/rollback transaction, before end the session, the resource is locked
- implicit transaction: skip above only write etl
- blocking happen when a session wants to work on a resource that is accessing by other session, e.g. 2 updates on the same table at the same time
- blocking is caused by poor transactional design/ long running transaction
- lock type:
    1. row lock: lock at individual row level
    2. page lock: lock at page level (a page is 8192 characters)
    3. entire table lock:
- SELECT * FROM sys.dm_tran_locks 
    - To view locks: request mode show what the second accessor wants to do
        - S: select
        - X: exclusive 
        - IX: intent exclusive
        - u: update
-   SELECT session_id, blocking_session_id, start_time, status, command, DB_NAME(database_id) as [database], wait_type, wait_resource, wait_time, open_transaction_count 

    FROM sys.dm_exec_requests 

    WHERE blocking_session_id > 0; 
    - To view blocking: 


### isolation level
-   SET TRANSACTION ISOLATION LEVEL  

    READ UNCOMMITTED/READ COMMITTED/REPEATABLE READ/SNAPSHOT/SERIALIZABLE 
    - READ UNCOMMITTED: completely ignore blocking, might cause dirty read(reading data that is not commited and could be rolled back)
    - READ COMMITTED:block depend on READ_COMMITTED_SNAPSHOT setting, can only read commited data
    - REPEATABLE READ:read remains the same until the end of the same transaction, no block, if:
        1. DB is not in recovery stats (restoring from backup)
        2. if having another alter db: ALTER DATABASE [DP300] SET ALLOW_SNAPSHOT_ISOLATION ON 
        - if insert has been done,  REPEATABLE return new data

    - SNAPSHOT: read remains the same until the end of the same transaction, no block
        - if insert has been done,  SNAPSHOT doesn't return new data,  only expanded version would be generated
    - SERIALIZABLE:block any dirty read, can't read stuff that is modified
         - block update and insert but not read
         - strictest, used in bank



- ALTER DATABASE [DP300] SET READ_COMMITTED_SNAPSHOT ON 
    - off is default for vm, on perm, when off read_commited is blocked
    - on is default for azure sql db, DML statement start generating row version that doesn't block read_commited 
- DBCC USEROPTIONS 
    - to see the current option

| Isolation level   | Dirty read | nonrepeatable read | phantom|
| :---------------- | :--------: |  :---------------: |:-----: |
| read uncommitted  |      v     |          v         |   v    |
| read committed    |            |          v         |   v    |
| repeated read     |            |                    |   v    |
| snapshot          |            |                    |        |
| serealizable      |            |                    |        |

- A phantom read occurs when a transaction performs a query with a WHERE clause (e.g., SELECT * FROM orders WHERE value > 1000), and then a second transaction inserts new rows that match the WHERE clause and commits. If the first transaction re-executes the same query, it might see more rows than it did initially. The original rows it read are still the same (repeatable read), but the set of rows has changed
    - a transaction end when it's explictly commited/rolled back/ client connection closed etc.
- Non-Repeatable Read: existing row always return same value

- Phantom Read: new rows might come in

### performance related db configuration
- this could be done in UI of ssms or tsql
- auto close: close when there is no connection , can't be enable in azure sql db
- auto_create_statistic: default on, allow generation info of content of each column, good for optimizer to decide for scan or seek
    -ALTER DATABASE [DP300]  SET AUTO_CREATE_STATISTICS OFF GO 
- auto_create_incremental_statistic: default off, collect even more thorough data than auto_create_statistic
- auto_shrink: default off, less burden to performance compared to auto_create_statistic but also less effective, not recommanded

- ``` 
    ALTER DATABASE SCOPED CONFIGURATION  
    -- [FOR SECONDARY] 
    SET GLOBAL_TEMPORARY_TABLE_AUTO_DROP = ON 
    -- LAST_QUERY_PLAN_STATS 
    -- LEGACY_CARDINALITY_ESTIMATION 
    -- MAXDOP 
    -- OPTIMIZE_FOR_AD_HOC_WORKLOADS 
    -- PARAMETER_SNIFFING 
    -- QUERY_OPTIMIZER_HOTFIXES 
    GO 
    ```
    - [FOR SECONDARY] if there is a geo-replicated db
    - GLOBAL_TEMPORARY_TABLE_AUTO_DROP drop GLOBAL_TEMPORARY_TABLE if not used in any session
    - LAST_QUERY_PLAN_STATS actual execution plan being shown or not in sys.db_exec_PLAN_STATS
    - LEGACY_CARDINALITY_ESTIMATION: how many row each value has in db, should only be used for compatibility purpose
    - MAXDOP: inter-query parallelism, max # of parellel thread, too high of the number migh impact performance, default 8 is usually a suitable #
    - OPTIMIZE_FOR_AD_HOC_WORKLOADS: do a compiled plan stub when a batch is compliled for first time, by the second time, only when compile second time a full compiled plan will be used, this reduce memory footprint
    - PARAMETER_SNIFFING : evluate stored procedure to create an execution plan, stick to the plan even parameter of stored procedure change, performance might be suboptimal
    - QUERY_OPTIMIZER_HOTFIXES: use the QUERY_OPTIMIZER_HOTFIXES ignoring compatibility

### Configure Intelligent Query Processing (IQP) 

- IQP is a set of features designed to improve query performance
- can be done using tsql or ssms ui
- first iop were included in sql server 2017 ( interleaved execution tvf & batch mode memory grant feedback), 2019 added a lot more
 
- SELECT * FROM sys.configurations 
    - Server-wide configuration options 
- EXEC sp_configure '101', 0
    - changing server configuration in azure sql mi or vm, not in db
- SELECT * FROM sys.database_scoped_configurations 
    - Database-wide configuration options
- ALTER DATABASE SCOPED CONFIGURATION  SET [config_name] = ON 
    - changing db configuration 
    - approx_count_distinct can't be disabled
- SELECT * FROM [schema].[table] OPTION (USE HINT ('config_name')) 
    - changing db configuration for one particular query


- adaptive join (batch mode)
    - it select between join or the nested looped join during runtime based on scanning actual first input rows
    - requirement:
        - col stored index in the query
        - or a table being referenced in the join
        - or batch mode enabled for row stored
- approx_count_distinct
    - approximation for aggragation of count distinct in each column, suitable for big data, decrease memory & performance requirement, 2 % error rate with 97% possibility
    - can't be disabled
- batch mode on row store
    - queries can work on batches of rows instead of 1 row at a time when cached, reduce db workload
    - happens by query plan decision, no need to set up
- interleaved execution tvf
    - it use the cardinality of multistatement table-valued-function (tvf) on first compilation instead of guessing, only for read only, e.g. select 
    - default fixed guess is 100 row
- batch mode memory grant feedback
    - there is also a row mode memory grant feedback
    - sql server look for how much memory allocated to a cached query, for the second run allocated same amount of memory to it, alternative is guessing memor, then adjust memory grant next time based on usage
- tsql scalar udf lining
    - user defined function, usually performa badly bcs they might run multiple time once per row bcs unable to work out performance cost
    - scalar udf transform into the equivalent related expression, it takes out a function put it into select (inlined) and increase performance
    - doesn't work with all function, especially for function with multiple return 
    - can be disabled for specific udf
- deffered compilation tv
    - it use the cardinality of multistatement table variable (tv) on first compilation instead of guessing, only for read only, e.g. select 
    - default fixed guess is 1 row


# lesson 15: automatic task and performance backup

### elastic job agent
- automate db maintenance task
- one could have a lot of db all over the place, some are azure sql db and some in the pool, with different server and subscription, for united actions on all these db (target group/ db of schard maps) e.g. backup, credential, collect performance data, telemetry data ( the automatic collection and transmission of data from remote sources), update reference data can be done through elastic job agent, as long as they are in the same azure cloud 
    - shard maps = how data is distributed across multiple databases (shards)
- in mi the equivalent is sql server agent job
- it charge for as a azure sql db, must be above basic tier, s0 can but isn't recommended, for frequent job even higher tier would be better
- an elastic job db is needed for an agent, a blank or azure sql db could be used to store logs, meta data , job related data, stored procedure 
- elastic job can execute (regardless of db is created before job agent):
    - against all db in a server
    - all db in a pool
    - single db
    - exclude specific db in target
- master db credential is needed to enumerate all db
    -  enumerate = the process of sequentially accessing each database in a group
- a job is a until of work which contains steps, each are tsql scripts and other details
- script must be Idempotent 
    - idempotent = operations produce the same result even when the operation is repeated many times
- then it'll generate output and job history, output could be saved as table, job history will be stored in job.job_executions for 45 days
- creation step:
    1. create azure sql db. minimum s0
    2. go to azure portal using ui/powershell create elastic job agent
    3. create db master key using tsql in ssms, password must be strong
        - CREATE MASTER KEY ENCRYPTION BY PASSWORD='<an6?%9++Vyd%Ut9';
    4. CREATE DATABASE SCOPED CREDENTIAL, need one to execute job,  one to refresh db metadata in the server
        - CREATE DATABASE SCOPED CREDENTIAL [MasterCred] WITH IDENTITY = 'MasterU', SECRET = 'an6?%9++Vyd%Ut9'
        - CREATE DATABASE SCOPED CREDENTIAL [RunJob] WITH IDENTITY = 'JobU', SECRET = 'an6?%9++Vyd%Ut9'
    5. Create a target group 
        - EXEC jobs.sp_add_target_group 'GrpDatabase';
    6. add a target group member 

        ´´´
        EXEC jobs.sp_add_target_group_member 
        @target_group_name = 'GrpDatabase', 
        @target_type = 'SqlDatabase',   -- or 'SqlServer' or 'PoolGroup' 
        -- @membership_type = 'Exclude', 
        -- @refresh_credential_name = 'RefreshPassword', 
        @server_name = 'dp300database.database.windows.net', 
        @database_name = 'dp300'; 
        ´´´
        - @membership_type = 'Exclude'  to exclude a certain computer
        - @refresh_credential_name = 'RefreshPassword' when target_type is a server or pool
    7. to create master user run this tsql In Master Database 
        ´´´
        CREATE LOGIN MasterU WITH PASSWORD ='<an6?%9++Vyd%Ut9' 
        CREATE USER MasterU FROM LOGIN MasterU 
        CREATE LOGIN JobU WITH PASSWORD = '<an6?%9++Vyd%Ut9' 
        ´´´
        - for the job to run we need 2 users, both with login in master db, one user in master db, one user in Target User db 
    8. to create user in Target User Database 
        ´´´
        CREATE USER JobU FROM LOGIN JobU 
        ALTER ROLE db_owner ADD MEMBER JobU
        ´´´
    9. Create a job and job steps in elastic job agent db
        ´´´
        EXEC jobs.sp_add_job @job_name='My first job', 
        @description='Look at objects' 
        EXEC jobs.sp_add_jobstep @job_name='My first job', 
        @command='SELECT * FROM sys.objects', 
        @credential_name='RunJob', 
        @target_group_name='GrpDatabase' 
        ´´´
    10. Run/schedule the job in T-SQL 
        - EXEC jobs.sp_start_job 'My first job' 
            - Run the job now

        ´´´
        EXEC jobs.sp_update_job 
        @job_name='My first job', 
        @enabled=1, 
        @schedule_interval_type='Minutes', -- Or Hours, Days, Weeks, Months or Once, 
        @schedule_interval_count=1 
        ´´´
            - schedule the job 
    11. Monitor job execution 
        - SELECT * FROM jobs.job_executions order by start_time 
        - or in azure portal seeing the last 100 job execution
    - adding target groups/credentials can only be done through tsql/powershell/visual studio, but not in azure portal

### log backup

- in azure sql db backup is done automatically
1. full backup 
    - is done every single week, same as in mi
2. differential backup
    - done every 12-24 hr, back up the current difference with last full backup
3. transactional backup
    - every 5 to 10 min backup everything since last backup(all 3 types)
- restore to point in time in last 7 day (option 1 to 35, default 7, basic only 7) with these backups is possible
- to restore to a point in time last full back up, last differential back up between that full backup and now, all transactional backup between that differential backup and now will be needed

- azure portal and powershell can both carry out a recover
- a new db is needed for restoration, restore over an existing db is not allowed
- restore db usually need to be on the same server
- to restore it to a different region, create a new db in the new region, in advanced using backup to populate the db

### longterm backup retention (ltr)
- available for azure sql mi and db
- the functionality is in public preview
- a backup for longer than 35 days
- only control by azure not admin
- might take up to 7 days before the first ltr backup
- secondary db should have ltr configured, otherwise failed over from primary db will lead to lose of ltr
- backups are stored in azure blob storage, a different storage container each week
- in configuration backup length for weekly, monthly, and yearly can be selected, but not timing




# lesson 16: high availability and disaster recovery (HADR) strategy for data platform

### service level agreement
- service level agreement (SLA) is what ms minimum guarantee, if ms failed to meet it, they'll refund
1. db
    - for hyperscale there is a min uptime for 99,9 % if no replica db, 99,95 % if 1 replica
    - other tiers  are 99.99 %replica
    - if configured for zone redundant deployment(db that have multiple synchronized in diff building of same region) + business critical/premium then 99,995%
2. mi
    - 99.99% (four 9)
3. vm
    - 95% - 99,99%
    - sql server on vm might fail on a healthy vm, so real sla is even lower for vm

### HADR strategy based on RPO/RTO
- geo-replica
    - setting can be configured under replica 
    - primary db send out data through asynchronous replication to replicas/secondaries, the process is called seeding. the update is replicated asynchronously, which means they are committed to primary before to secondary
    - secondary need to be at least same service tier as primary, while upgrading, upgrade secondary first then primary, only when changing from general to business critical need to disconnect the 2
    - up tp 4 replicas possible
    - building replica for replica is possible
    - secondary can be read, so using primary for write and secondary for read can help sharing the workload
    - after failure, one of the secondary can be appointed as new primary and the other secondary will be connected to this new primary, but the connection string will need to be updated manually
- failover group
    - with failover group, azure will manage the connection string for a group of db, once primary is failed, it switch to a secondary without changing the string
    - if choosing automatic failover policy, manual failover will still be available
|                                    | Geo replica | failover group |
| :--------------------------------: | :-: | :-: |
| automatic failover                 |  x  |  v  |
| failover multiple db simultaneously|  x  |  v  |
| sql mi support                     |  x  |  v  |
| user update connection string after failover (time consuming) |  v   | x |
| can be in same region as primary   |  v  |  x  |
| multiple replica                   |  v  |  x  |
| support read-scale                 |  v  |  v  |

- recovery point objective (RPO) = how much data one could lose 
- recovery time objective (RTO) = max failover time, during rto no backup will happen
- with a geo replication,
    - manual db failover, rpo is 5 sec, rto is 30 sec
    - auto failover group, rpo is 5 sec, rto is 1 hr minimum
- for a geo-restore from geo replicated backup, rpo is 1 hr, rto is 12 hr
- the shorter the fail timeframe, the more expensive it cost


# lesson 17 perform administrative using tsql
### 